<a href="https://colab.research.google.com/github/GeorgeM2000/CANE/blob/master/code/CANE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Libraries & Tools***

In [ ]:
import argparse
import os
import random
import numpy as np
import tensorflow as tf
from math import pow
from datetime import datetime
from pathlib import Path
import shutil
from tqdm import tqdm
import zipfile
import tqdm

In [ ]:
!pwd

# ***Global Variables***

In [ ]:
MAX_LEN = 300 # Default value
neg_table_size = 1000000
NEG_SAMPLE_POWER = 0.75
batch_size = 64
num_epoch = 200 # Default: 200
embed_size = 200
lr = 1e-3

In [ ]:
datasetName = "cora"
dataTextFile = "data.txt"
ratio = 0.95
rho = "1.0,0.3,0.3"

In [ ]:
max_word_count = 0
min_word_count = float('inf')

with open(f'/content/datasets/{datasetName}/{dataTextFile}', 'r') as file:
    for line in file:
        word_count = len(line.split())

        if word_count > max_word_count:
            max_word_count = word_count

        if word_count < min_word_count:
            min_word_count = word_count

print("Max word count:", max_word_count)
print("Min word count:", min_word_count)


MAX_LEN = max_word_count + 1

Max word count: 473
Min word count: 10


In [ ]:
zip_file_path = '/content/data.zip'
extract_to = f'/content/datasets/{datasetName}'

# Open and extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete!")

Extraction complete!


# ***DataSet***

In [ ]:
class dataSet:
    def __init__(self, text_path, graph_path):

        text_file, graph_file = self.load(text_path, graph_path)

        self.edges = self.load_edges(graph_file)

        self.text, self.num_vocab, self.num_nodes = self.load_text(text_file)

        self.negative_table = InitNegTable(self.edges)

    def load(self, text_path, graph_path):
        text_file = open(text_path, 'rb').readlines()
        for a in range(0, len(text_file)):
            text_file[a] = str(text_file[a])
        graph_file = open(graph_path, 'rb').readlines()

        return text_file, graph_file

    def load_edges(self, graph_file):
        edges = []
        for i in graph_file:
            edges.append(list(map(int, i.strip().decode().split('\t'))))

        print("Total load %d edges." % len(edges))

        return edges

    def load_text(self, text_file):
        """
        Adapting with adapt(text_data):

        vectorize_layer.adapt(text_data) analyzes text_data, builds a vocabulary, and assigns a unique integer ID to each word based on its frequency (most frequent words get lower IDs).
        Transforming with vectorize_layer(text_data):

        This maps each word in text_data to its corresponding integer token ID, producing a 2D array where each row represents a sequence of token IDs for a given input line, padded or truncated to max_len.
        """
        vectorize_layer = tf.keras.layers.TextVectorization(
            max_tokens=None,  # Set a limit if needed
            output_mode='int',
            output_sequence_length=MAX_LEN
        )

        text_data = [line.strip() for line in text_file]

        vectorize_layer.adapt(text_data)

        text = vectorize_layer(text_data).numpy()

        num_vocab = len(vectorize_layer.get_vocabulary())
        print(f'Vocabulary: {num_vocab}')
        num_nodes = len(text)

        return text, num_vocab, num_nodes

    def negative_sample(self, edges):
        node1, node2 = zip(*edges)
        sample_edges = []
        func = lambda: self.negative_table[random.randint(0, neg_table_size - 1)]
        for i in range(len(edges)):
            neg_node = func()
            while node1[i] == neg_node or node2[i] == neg_node:
                neg_node = func()
            sample_edges.append([node1[i], node2[i], neg_node])

        return sample_edges

    def generate_batches(self, mode=None):

        num_batch = len(self.edges) // batch_size
        edges = self.edges
        # if mode == 'add':
        #     num_batch += 1
        #     edges.extend(edges[:(config.batch_size - len(self.edges) // config.batch_size)])
        if mode != 'add':
            random.shuffle(edges)
        sample_edges = edges[:num_batch * batch_size]
        sample_edges = self.negative_sample(sample_edges)

        batches = []
        for i in range(num_batch):
            batches.append(sample_edges[i * batch_size:(i + 1) * batch_size])
        # print sample_edges[0]
        return batches


# ***CANE***

In [ ]:
class Model:
    def __init__(self, vocab_size, num_nodes, rho):
        rho = rho.split(",")
        self.rho1 = float(rho[0])
        self.rho2 = float(rho[1])
        self.rho3 = float(rho[2])
        # '''hyperparameter'''
        with tf.name_scope('read_inputs') as scope:
          self.Text_a = tf.compat.v1.placeholder(tf.int32, [batch_size, MAX_LEN], name='Ta')
          self.Text_b = tf.compat.v1.placeholder(tf.int32, [batch_size, MAX_LEN], name='Tb')
          self.Text_neg = tf.compat.v1.placeholder(tf.int32, [batch_size, MAX_LEN], name='Tneg')
          self.Node_a = tf.compat.v1.placeholder(tf.int32, [batch_size], name='n1')
          self.Node_b = tf.compat.v1.placeholder(tf.int32, [batch_size], name='n2')
          self.Node_neg = tf.compat.v1.placeholder(tf.int32, [batch_size], name='n3')

        with tf.name_scope('initialize_embedding') as scope:
            self.text_embed = tf.Variable(tf.random.truncated_normal([vocab_size, embed_size // 2], stddev=0.3))
            self.node_embed = tf.clip_by_norm(tf.Variable(tf.random.truncated_normal([num_nodes, embed_size // 2], stddev=0.3)), clip_norm=1, axes=1)

        with tf.name_scope('lookup_embeddings') as scope:
            self.TA = tf.nn.embedding_lookup(self.text_embed, self.Text_a)
            self.T_A = tf.expand_dims(self.TA, -1)

            self.TB = tf.nn.embedding_lookup(self.text_embed, self.Text_b)
            self.T_B = tf.expand_dims(self.TB, -1)

            self.TNEG = tf.nn.embedding_lookup(self.text_embed, self.Text_neg)
            self.T_NEG = tf.expand_dims(self.TNEG, -1)

            self.N_A = tf.nn.embedding_lookup(self.node_embed, self.Node_a)
            self.N_B = tf.nn.embedding_lookup(self.node_embed, self.Node_b)
            self.N_NEG = tf.nn.embedding_lookup(self.node_embed, self.Node_neg)

        self.convA, self.convB, self.convNeg = self.conv()
        self.loss = self.compute_loss()

    def conv(self):
        W2 = tf.Variable(tf.random.truncated_normal([2, embed_size // 2, 1, 100], stddev=0.3))
        rand_matrix = tf.Variable(tf.random.truncated_normal([100, 100], stddev=0.3))

        convA = tf.nn.conv2d(self.T_A, W2, strides=[1, 1, 1, 1], padding='VALID')
        convB = tf.nn.conv2d(self.T_B, W2, strides=[1, 1, 1, 1], padding='VALID')
        convNEG = tf.nn.conv2d(self.T_NEG, W2, strides=[1, 1, 1, 1], padding='VALID')

        hA = tf.tanh(tf.squeeze(convA))
        hB = tf.tanh(tf.squeeze(convB))
        hNEG = tf.tanh(tf.squeeze(convNEG))

        tmphA = tf.reshape(hA, [batch_size * (MAX_LEN - 1), embed_size // 2])
        ha_mul_rand = tf.reshape(tf.matmul(tmphA, rand_matrix),
                                 [batch_size, MAX_LEN - 1, embed_size // 2])
        r1 = tf.matmul(ha_mul_rand, hB, adjoint_b=True)
        r3 = tf.matmul(ha_mul_rand, hNEG, adjoint_b=True)
        att1 = tf.expand_dims(tf.stack(r1), -1)
        att3 = tf.expand_dims(tf.stack(r3), -1)

        att1 = tf.tanh(att1)
        att3 = tf.tanh(att3)

        pooled_A = tf.reduce_mean(att1, 2)
        pooled_B = tf.reduce_mean(att1, 1)
        pooled_NEG = tf.reduce_mean(att3, 1)

        a_flat = tf.squeeze(pooled_A)
        b_flat = tf.squeeze(pooled_B)
        neg_flat = tf.squeeze(pooled_NEG)

        w_A = tf.nn.softmax(a_flat)
        w_B = tf.nn.softmax(b_flat)
        w_NEG = tf.nn.softmax(neg_flat)

        rep_A = tf.expand_dims(w_A, -1)
        rep_B = tf.expand_dims(w_B, -1)
        rep_NEG = tf.expand_dims(w_NEG, -1)

        hA = tf.transpose(hA, perm=[0, 2, 1])
        hB = tf.transpose(hB, perm=[0, 2, 1])
        hNEG = tf.transpose(hNEG, perm=[0, 2, 1])

        rep1 = tf.matmul(hA, rep_A)
        rep2 = tf.matmul(hB, rep_B)
        rep3 = tf.matmul(hNEG, rep_NEG)

        attA = tf.squeeze(rep1)
        attB = tf.squeeze(rep2)
        attNEG = tf.squeeze(rep3)

        return attA, attB, attNEG

    def compute_loss(self):
        p1 = tf.reduce_sum(tf.multiply(self.convA, self.convB), 1)
        p1 = tf.math.log(tf.nn.sigmoid(p1) + 0.001)

        p2 = tf.reduce_sum(tf.multiply(self.convA, self.convNeg), 1)
        p2 = tf.math.log(tf.nn.sigmoid(-p2) + 0.001)

        p3 = tf.reduce_sum(tf.multiply(self.N_A, self.N_B), 1)
        p3 = tf.math.log(tf.nn.sigmoid(p3) + 0.001)

        p4 = tf.reduce_sum(tf.multiply(self.N_A, self.N_NEG), 1)
        p4 = tf.math.log(tf.nn.sigmoid(-p4) + 0.001)

        p5 = tf.reduce_sum(tf.multiply(self.convB, self.N_A), 1)
        p5 = tf.math.log(tf.nn.sigmoid(p5) + 0.001)

        p6 = tf.reduce_sum(tf.multiply(self.convNeg, self.N_A), 1)
        p6 = tf.math.log(tf.nn.sigmoid(-p6) + 0.001)

        p7 = tf.reduce_sum(tf.multiply(self.N_B, self.convA), 1)
        p7 = tf.math.log(tf.nn.sigmoid(p7) + 0.001)

        p8 = tf.reduce_sum(tf.multiply(self.N_B, self.convNeg), 1)
        p8 = tf.math.log(tf.nn.sigmoid(-p8) + 0.001)

        rho1 = self.rho1
        rho2 = self.rho2
        rho3 = self.rho3
        temp_loss = rho1 * (p1 + p2) + rho2 * (p3 + p4) + rho3 * (p5 + p6) + rho3 * (p7 + p8)
        loss = -tf.reduce_sum(temp_loss)
        return loss

# ***Negative Sample***

In [ ]:
def InitNegTable(edges):
    a_list, b_list = zip(*edges)
    a_list = list(a_list)
    b_list = list(b_list)
    node = a_list
    node.extend(b_list)

    node_degree = {}
    for i in node:
        if i in node_degree:
            node_degree[i] += 1
        else:
            node_degree[i] = 1
    sum_degree = 0
    for i in node_degree.values():
        sum_degree += pow(i, 0.75)

    por = 0
    cur_sum = 0
    vid = -1
    neg_table = []
    degree_list = list(node_degree.values())
    node_id = list(node_degree.keys())
    for i in range(neg_table_size):
        if ((i + 1) / float(neg_table_size)) > por:
            cur_sum += pow(degree_list[vid + 1], NEG_SAMPLE_POWER)
            por = cur_sum / sum_degree
            vid += 1
        neg_table.append(node_id[vid])
    print(len(neg_table))
    return neg_table


# ***Run (Single execution)***

In [ ]:
def prepareData(datasetName, ratio):
  f = open('/content/datasets/%s/graph.txt' % datasetName, 'rb')
  edges = [i for i in f]
  selected = int(len(edges) * float(ratio))
  selected = selected - selected % batch_size
  selected = random.sample(edges, selected)
  remain = [i for i in edges if i not in selected]
  try:
    temp_dir = Path('temp')

    # Check if the directory exists, if so, delete it
    if temp_dir.exists() and temp_dir.is_dir():
        shutil.rmtree(temp_dir)
        print("Existing directory deleted.")

    # Create the directory
    temp_dir.mkdir(parents=True, exist_ok=True)
    print("Directory created successfully.")

  except Exception as e:
      print(f"An error occurred: {e}")

  fw1 = open('temp/graph.txt', 'wb')
  fw2 = open('temp/test_graph.txt', 'wb')

  for i in selected:
      fw1.write(i)
  for i in remain:
      fw2.write(i)

In [ ]:
prepareData(datasetName, ratio)

Existing directory deleted.
Directory created successfully.


In [ ]:
# load data
dataset_name = datasetName
graph_path = os.path.join('/content/temp/graph.txt')
text_path = os.path.join("/content", "datasets", dataset_name, dataTextFile)

data = dataSet(text_path, graph_path)

Total load 4928 edges.
Vocabulary: 16169
1000000


In [ ]:
with tf.Graph().as_default():
    sess = tf.compat.v1.Session()
    with sess.as_default():
        model = Model(data.num_vocab, data.num_nodes, rho)
        opt = tf.compat.v1.train.AdamOptimizer(lr)#tf.keras.optimizers.Adam(learning_rate=lr)
        train_op = opt.minimize(model.loss)#opt.minimize(model.loss, var_list=model.trainable_variables)
        sess.run(tf.compat.v1.global_variables_initializer())
        time = 0

        # training
        print('start training.......')


        for epoch in tqdm(range(num_epoch), desc="Epochs"):
            start = datetime.now()
            loss_epoch = 0
            batches = data.generate_batches()
            h1 = 0
            num_batch = len(batches)
            for i in range(num_batch):
                batch = batches[i]

                node1, node2, node3 = zip(*batch)
                node1, node2, node3 = np.array(node1), np.array(node2), np.array(node3)
                text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]

                feed_dict = {
                    model.Text_a: text1,
                    model.Text_b: text2,
                    model.Text_neg: text3,
                    model.Node_a: node1,
                    model.Node_b: node2,
                    model.Node_neg: node3
                }

                # run the graph
                _, loss_batch = sess.run([train_op, model.loss], feed_dict=feed_dict)

                loss_epoch += loss_batch

            end = datetime.now()
            time += (end - start).total_seconds()
            print('epoch: ', epoch + 1, ' loss: ', loss_epoch)

        print(f'Time: {time}')
        # Saving embeddings
        with open('temp/embed.txt', 'wb') as file:
            batches = data.generate_batches(mode='add')
            num_batch = len(batches)
            embed = [[] for _ in range(data.num_nodes)]

            for i in range(num_batch):
                batch = batches[i]
                node1, node2, node3 = zip(*batch)
                node1, node2, node3 = np.array(node1), np.array(node2), np.array(node3)
                text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]

                feed_dict = {
                    model.Text_a: text1,
                    model.Text_b: text2,
                    model.Text_neg: text3,
                    model.Node_a: node1,
                    model.Node_b: node2,
                    model.Node_neg: node3
                }

                # Fetch embeddings
                #convA, convB, TA, TB = sess.run([model.convA, model.convB, model.N_A, model.N_B])
                convA, convB, TA, TB = sess.run([model.convA, model.convB, model.N_A, model.N_B], feed_dict=feed_dict)

                for j in range(batch_size):
                    #em = list(convA[j]) + list(TA[j])
                    em = list(TA[j])
                    embed[node1[j]].append(em)
                    #em = list(convB[j]) + list(TB[j])
                    em = list(TB[j])
                    embed[node2[j]].append(em)


            for i in range(data.num_nodes):
                if embed[i]:
                    tmp = np.mean(embed[i], axis=0)
                    file.write((' '.join(map(str, tmp)) + '\n').encode())
                else:
                    file.write('\n'.encode())

start training.......


Epochs:   0%|          | 1/200 [00:03<12:32,  3.78s/it]

epoch:  1  loss:  17824.984832763672


Epochs:   1%|          | 2/200 [00:04<07:22,  2.24s/it]

epoch:  2  loss:  12845.764465332031


...

epoch:  198  loss:  5189.759601593018


Epochs: 100%|█████████▉| 199/200 [04:02<00:01,  1.20s/it]

epoch:  199  loss:  5076.978076934814


Epochs: 100%|██████████| 200/200 [04:03<00:00,  1.22s/it]

epoch:  200  loss:  5090.823783874512
Time: 242.84591500000002


In [ ]:
node2vec = {}
f = open('temp/embed.txt', 'rb')
for i, j in enumerate(f):
    if j.decode() != '\n':
        node2vec[i] = list(map(float, j.strip().decode().split(' ')))
f1 = open(os.path.join('temp/test_graph.txt'), 'rb')
edges = [list(map(int, i.strip().decode().split('\t'))) for i in f1]
nodes = list(set([i for j in edges for i in j]))
a = 0
b = 0
for i, j in edges:
    if i in node2vec.keys() and j in node2vec.keys():
        dot1 = np.dot(node2vec[i], node2vec[j])
        random_node = random.sample(nodes, 1)[0]
        while random_node == j or random_node not in node2vec.keys():
            random_node = random.sample(nodes, 1)[0]
        dot2 = np.dot(node2vec[i], node2vec[random_node])
        if dot1 > dot2:
            a += 1
        elif dot1 == dot2:
            a += 0.5
        b += 1

print("Auc value:", float(a) / b)

Auc value: 0.9501915708812261


# ***Run (Multiple executions)***

In [ ]:
graph_paths = ['/path/to/graph1.txt']
text_paths = ['/path/to/text1.txt', '/path/to/text2.txt']

# Log file to save execution details
log_file = 'CANE_execution_logs.txt'

# Load data and execute for each combination of graph_path and text_path
for graph_path in graph_paths:
    for text_path in tqdm(text_paths, desc="Processing Text Files"):
        data = dataSet(text_path, graph_path)

        # Logging the execution details
        with open(log_file, 'a') as log:
            log.write(f'Processing graph_path: {graph_path}, text_path: {text_path}\n')

        with tf.Graph().as_default():
            sess = tf.compat.v1.Session()
            with sess.as_default():
                model = Model(data.num_vocab, data.num_nodes, rho)
                opt = tf.compat.v1.train.AdamOptimizer(lr)
                train_op = opt.minimize(model.loss)
                sess.run(tf.compat.v1.global_variables_initializer())
                #total_time = 0

                # Training
                start_time = datetime.now()
                print('Start training...')
                for epoch in tqdm(range(num_epoch), desc="Epochs"):
                    #start_time = datetime.now()
                    loss_epoch = 0
                    batches = data.generate_batches()
                    num_batch = len(batches)

                    for i in range(num_batch):
                        batch = batches[i]
                        node1, node2, node3 = zip(*batch)
                        node1, node2, node3 = np.array(node1), np.array(node2), np.array(node3)
                        text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]

                        feed_dict = {
                            model.Text_a: text1,
                            model.Text_b: text2,
                            model.Text_neg: text3,
                            model.Node_a: node1,
                            model.Node_b: node2,
                            model.Node_neg: node3
                        }

                        # Run the graph
                        _, loss_batch = sess.run([train_op, model.loss], feed_dict=feed_dict)
                        loss_epoch += loss_batch

                    #end_time = datetime.now()
                    #epoch_time = (end_time - start_time).total_seconds()
                    #total_time += epoch_time

                end_time = datetime.now()
                # Log epoch and loss
                with open(log_file, 'a') as log:
                    log.write(f'Loss: {loss_epoch}, Time: {(end_time - start_time).total_seconds()}\n')

                #print(f'Total Time: {total_time} seconds')

                # Save embeddings with a unique name
                embed_file = f'temp/embed_{os.path.basename(graph_path)}_{os.path.basename(text_path)}.txt'
                with open(embed_file, 'wb') as file:
                    batches = data.generate_batches(mode='add')
                    num_batch = len(batches)
                    embed = [[] for _ in range(data.num_nodes)]

                    for i in range(num_batch):
                        batch = batches[i]
                        node1, node2, node3 = zip(*batch)
                        node1, node2, node3 = np.array(node1), np.array(node2), np.array(node3)
                        text1, text2, text3 = data.text[node1], data.text[node2], data.text[node3]

                        feed_dict = {
                            model.Text_a: text1,
                            model.Text_b: text2,
                            model.Text_neg: text3,
                            model.Node_a: node1,
                            model.Node_b: node2,
                            model.Node_neg: node3
                        }

                        # Fetch embeddings
                        convA, convB, TA, TB = sess.run(
                            [model.convA, model.convB, model.N_A, model.N_B],
                            feed_dict=feed_dict
                        )

                        for j in range(batch_size):
                            em = list(TA[j])
                            embed[node1[j]].append(em)
                            em = list(TB[j])
                            embed[node2[j]].append(em)

                    for i in range(data.num_nodes):
                        if embed[i]:
                            tmp = np.mean(embed[i], axis=0)
                            file.write((' '.join(map(str, tmp)) + '\n').encode())
                        else:
                            file.write('\n'.encode())

                # Log completion
                with open(log_file, 'a') as log:
                    log.write(f'Embeddings saved to: {embed_file}\n')


In [ ]:
embed_files = ["embed1.txt", "embed2.txt", "embed3.txt"]
test_graph_file = "temp/test_graph.txt"

# Initialize a log file to store the AUC results
with open("CANE_auc_results.log", "w") as auc_file:
    auc_file.write("Embed File\tAUC Value\n")

# Loop through each embed.txt file
for embed_file in tqdm(embed_files, desc="Processing embed files"):
    node2vec = {}

    # Load the embeddings from the current embed file
    with open(embed_file, 'rb') as f:
        for i, j in enumerate(f):
            if j.decode().strip():
                node2vec[i] = list(map(float, j.strip().decode().split(' ')))

    # Load the edges from the test graph file
    with open(test_graph_file, 'rb') as f1:
        edges = [list(map(int, i.strip().decode().split())) for i in f1]
    nodes = list(set([i for j in edges for i in j]))

    # Calculate AUC
    a = 0
    b = 0
    for i, j in edges:
        if i in node2vec.keys() and j in node2vec.keys():
            dot1 = np.dot(node2vec[i], node2vec[j])
            random_node = random.sample(nodes, 1)[0]
            while random_node == j or random_node not in node2vec.keys():
                random_node = random.sample(nodes, 1)[0]
            dot2 = np.dot(node2vec[i], node2vec[random_node])
            if dot1 > dot2:
                a += 1
            elif dot1 == dot2:
                a += 0.5
            b += 1

    auc_value = float(a) / b if b > 0 else 0
    print(f"AUC value for {embed_file}: {auc_value}")

    # Log the result
    with open("CANE_auc_results.log", "a") as auc_file:
        auc_file.write(f"{embed_file}\t{test_graph_file}\t{auc_value}\n")